In [1]:
# 웹툰 titleid 리스트, 각 웹툰 화 개수 리스트
titles = ['여름의 너에게','서울시 천사주의']
titleids = [799220,799478]
episode_counts = [44,49]

In [2]:
# !pip install --upgrade selenium
# !pip show selenium
# !pip install openpyxl
# !pip install --upgrade openpyxl
import os
from selenium import webdriver
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import Workbook
from selenium.webdriver.chrome.options import Options


In [3]:
# 페이지에서 더보기 버튼이 안나올때까지 버튼 누르고, 댓글 추출 함수
def Click_Button_Collect_Review(title,titleid, episode, driver,df):
    driver.get('https://comic.naver.com/webtoon/detail?titleId={}&no={}'.format(titleid,episode))
    driver.find_element(By.XPATH,'//*[@id="cbox_module"]/div/div[8]/a').click()
    while True :
        try:
            view_more_button = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="cbox_module"]/div/div[7]/a')))
            view_more_button.click()
            time.sleep(1)
        except Exception as e :
            break
            
    review_tags = driver.find_elements(By.XPATH,'//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul/li/div[1]/div/div[2]/span[1]')
    
    episode_review_df = pd.DataFrame(columns=['title','episode','review'])
    for review_tag in review_tags :
        row = []
        comment = review_tag.text
        row = [title,episode,comment]
        
        df_row = pd.DataFrame(data = [row], columns=['title','episode','review'])
        
        episode_review_df = pd.concat([episode_review_df,df_row]) 
    return episode_review_df

In [6]:
# 드라이버 열고 웹툰의 개별 화에 접속하기위한 titleid와 episode 정보 정의

driver = webdriver.Chrome()
driver.implicitly_wait(5)
i = 0

for r in range(0,len(titleids)):
    titleid = titleids[r]
    title = titles[r]
    df = pd.DataFrame(columns=['title','episode','review'])
    
    try:
        for episode in range(1,3):
            episode_review = Click_Button_Collect_Review(title, titleid, episode, driver,df)
            df=pd.concat([df,episode_review])
            print('{}화 완성'.format(episode))
    finally:
        df.to_excel('{}.xlsx'.format(title), sheet_name='{}화'.format(episode_counts[r]+1))
    
driver.quit()

1화 완성
2화 완성
1화 완성
2화 완성
